<a href="https://colab.research.google.com/github/gtzambranop/UN_2020_2/blob/DCD/DCD/module_2_francisco/Clase_8_Python_b%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo práctico

### Lenet5


<img src="lenet5.png" alt="Drawing" style="width: 800px;"/>

[Paper Lenet5](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf)

### Carga de módulos

In [ ]:
#Carga de módulos
from keras.models import Sequential, load_model
from keras import layers
from keras.utils import to_categorical
from keras.datasets import fashion_mnist
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from datetime import datetime as dt
import numpy as np

### Carga de Mnist-fashion

In [ ]:
dc_labels = {0: 'T-shirt/top',
          1: 'Trouser',
          2: 'Pullover',
          3: 'Dress',
          4: 'Coat',
          5: 'Sandal',
          6: 'Shirt',
          7: 'Sneaker',
          8: 'Bag',
          9: 'Ankle boot'}

In [ ]:
# Carga del conjunto de datos
(X_train, y_train_lab), (X_test, y_test_lab) = fashion_mnist.load_data()

# Escalamiento del conjunto de datos al intervalo [0,1]
X_train = X_train / 255
X_test = X_test / 255

# Redimensionamiento del conjunto de datos
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

# Convirtiendo la etiqueta en una variable dummy
y_train = to_categorical(y_train_lab)
y_test = to_categorical(y_test_lab)

### Explorando una sola imagen

In [ ]:
n = 2
print(y_train_lab[n])
#print(dc_labels[y_train_lab[n]])
#print(y_train[n])
#print(X_train[n,:,:,:])
#plt.imshow(X_train[n,:,:,:].reshape(28,28), cmap='gray_r')

### Validando múltiples imágenes

In [ ]:
def plot_images_sample(X, y, predict=False):
    # Numero de imagenes
    n_images = 9 
    
    # Selección aleatoria
    index = np.random.choice(np.arange(len(X)), size=n_images, replace = False)
    print('Indices de selección aleatoria: {}'.format(index))
    
    # Extracción de imagenes a imprimir
    X_plot = X[index, : ]
    y_plot = y[index]
    
    # Creación de grilla para el gráfico
    fig, axes = plt.subplots(3,3, 
                         figsize=(5,5),
                         sharex=True, sharey=True,
                         subplot_kw=dict(aspect='equal'))
    
    # Se imprime cada imágen en un cuadro de la grilla
    for i, image in enumerate(X_plot):
        row = i//3 
        col = i%3  
        ax = axes[row, col]
        img_plot = np.reshape(image, (28,28))
        ax.imshow(img_plot, cmap='gray_r')
        if predict:
            text = 'Predict: {}'
        else:
            text = 'Label: {}'
        ax.set_title(text.format(dc_labels[y_plot[i]]))
        ax.set_xbound([0,28])    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_images_sample(X_train, y_train_lab)

### Definición del modelo
[Keras models](https://keras.io/api/models/)
[Layer Conv2d](https://keras.io/api/layers/convolution_layers/convolution2d/)
[Layer Averagepooling2D](https://keras.io/api/layers/pooling_layers/average_pooling2d/)

In [ ]:
# Secuential 
model = Sequential()

# Capa de convolución que no reduce dimensiones
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', 
                        input_shape=(28,28,1), padding='same'))

# Capa de pooling que calcula promedio y reduce dimensiones a la mitad
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

# Capa de convolución que usa 16 kernels reduce dimensiones de 14x14 a 10x10
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))

# Capa de pooling que calcula promedio y reduce dimensiones a la mitad
model.add(layers.AveragePooling2D())

# Aplanamiento de imagenes a un vector de 400 elementos
model.add(layers.Flatten())

# Reducción de dimensiones del vector
model.add(layers.Dense(units=120, activation='relu'))
model.add(layers.Dense(units=84, activation='relu'))

# La capa softmax calcula la probabilidad que un elemento pertenezca a una cierta categoría
model.add(layers.Dense(units=10, activation = 'softmax'))

### Resumen del modelo

In [ ]:
model.summary()

### Entrenamiento del modelo

In [ ]:
# Se compila el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

start = dt.now()
# Se entrena el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=8, batch_size=64)
print(dt.now() - start)

### Controlando el sobreajuste

<img src="overfit.png" alt="Drawing" style="width: 500px;"/>

[Regularizadores Keras](https://keras.io/api/layers/regularization_layers/)

### Haciendo predicciones con el modelo

In [ ]:
pred = model.predict(X_test)
pred_lab = np.array([np.argmax(x) for x in pred])
print(pred[0])
#print(pred_lab[0])

### Visualización de las predicciones

In [ ]:
plot_images_sample(X_test, pred_lab, predict=True)

### Guardar modelo


In [ ]:
model.save('MNist_fashion_Keras.h5')

### Cargar modelo

In [ ]:
model_loaded = load_model('MNist_fashion_Keras.h5')
model_loaded.summary()

### Predicciones sobre un modelo cargado

In [ ]:
pred = model_loaded.predict(X_test)
pred_lab = np.array([np.argmax(x) for x in pred])
plot_images_sample(X_test, pred_lab)